# Let's try out a neural network

## Simplified Data

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# from fastbook import *
# from utils import *
from fastai.tabular.all import *
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.tree import DecisionTreeClassifier
# #from dtreeviz.trees import *
# from IPython.display import Image, display_svg, SVG
# from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
# from sklearn.metrics import accuracy_score, roc_auc_score 
from sklearn.metrics import roc_auc_score 


path = Path('/Users/baranserajelahi/Codes/fraud-detection-pytorch-scikit-fastai/data')
Path.BASE_PATH = path
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 300

In [3]:
xs_final = (path/'xs_final.pkl').load()
dep_var = 'isFraud'

In [4]:
to = (path/'to.pkl').load()

In [5]:
df_nn = to[list(xs_final.columns) + [dep_var]]

In [6]:
assert len(xs_final.columns)==(len(df_nn.columns)-1)

In [7]:
df_nn.head()

,TransactionDT,TransactionAmt,card1,card2,C13,D3,addr1,D2,C1,C14,D1,D15,C2,C5,D5,D4,P_emaildomain,D10,card5,C11,C6,M4,dist1,C8,D11,R_emaildomain,card6,M5,C12,D8,C9,id_02,id_31,id_19,card4,M6,id_17,ProductCD,id_13,DeviceInfo,C7,card3,id_01,D2_na,M9,M7,DeviceType,D9,D3_na,id_06,addr2,M2,D5_na,id_33,id_05,D4_na,dist2,dist1_na,D6,id_37,id_02_na,id_14,id_18,D12,D11_na,D15_na,isFraud
57195,1316341,418.950012,1233,390,2.0,463.0,68,97.0,2.0,2.0,472.0,288.0,1.0,2.0,288.0,288.0,3,288.0,70,1.0,1.0,0,8.0,0.0,36.0,0,3,0,0.0,39.208332,1.0,123602.0,0,0,4,2,0,5,0,0,0.0,30,-5.0,2,0,0,0,0.666666,1,0.0,21,0,1,0,0.0,1,36.0,2,0.0,0,2,0,0,0.0,2,1,0
312877,7780176,59.000000,3740,168,12.0,8.0,70,97.0,5.0,4.0,0.0,330.0,3.0,1.0,196.0,330.0,17,400.0,40,1.0,2.0,1,4.0,0.0,301.0,0,3,1,0.0,39.208332,1.0,123602.0,0,0,4,2,0,5,0,0,0.0,30,-5.0,2,2,1,0,0.666666,2,0.0,21,2,1,0,0.0,1,36.0,1,0.0,0,2,0,0,0.0,1,1,0
195195,4394717,29.000000,1196,317,8.0,0.0,64,51.0,1.0,1.0,51.0,51.0,2.0,0.0,0.0,51.0,54,51.0,51,2.0,1.0,1,1.0,0.0,51.0,0,3,2,0.0,39.208332,2.0,123602.0,0,0,4,1,0,5,0,0,0.0,30,-5.0,1,2,1,0,0.666666,1,0.0,21,2,1,0,0.0,1,36.0,1,0.0,0,2,0,0,0.0,1,1,1
21600,572518,24.000000,2236,94,1.0,8.0,6,97.0,1.0,1.0,0.0,0.0,1.0,0.0,9.0,24.0,0,0.0,57,1.0,0.0,2,0.0,0.0,0.0,0,3,2,0.0,39.208332,1.0,123602.0,0,0,3,1,0,5,0,0,0.0,30,-5.0,2,0,0,0,0.666666,2,0.0,21,2,2,0,0.0,2,36.0,1,0.0,0,2,0,0,0.0,1,1,0
12253,348181,57.950001,3916,115,18.0,7.0,26,484.0,2.0,2.0,484.0,48.0,2.0,3.0,9.0,24.0,0,483.0,70,3.0,1.0,0,27.0,0.0,36.0,0,3,0,0.0,39.208332,2.0,123602.0,0,0,4,1,0,5,0,0,0.0,30,-5.0,1,0,0,0,0.666666,1,0.0,21,2,2,0,0.0,2,36.0,1,0.0,0,2,0,0,0.0,2,2,0


In [8]:
df_nn.columns

Index(['TransactionDT', 'TransactionAmt', 'card1', 'card2', 'C13', 'D3',
       'addr1', 'D2', 'C1', 'C14', 'D1', 'D15', 'C2', 'C5', 'D5', 'D4',
       'P_emaildomain', 'D10', 'card5', 'C11', 'C6', 'M4', 'dist1', 'C8',
       'D11', 'R_emaildomain', 'card6', 'M5', 'C12', 'D8', 'C9', 'id_02',
       'id_31', 'id_19', 'card4', 'M6', 'id_17', 'ProductCD', 'id_13',
       'DeviceInfo', 'C7', 'card3', 'id_01', 'D2_na', 'M9', 'M7', 'DeviceType',
       'D9', 'D3_na', 'id_06', 'addr2', 'M2', 'D5_na', 'id_33', 'id_05',
       'D4_na', 'dist2', 'dist1_na', 'D6', 'id_37', 'id_02_na', 'id_14',
       'id_18', 'D12', 'D11_na', 'D15_na', 'isFraud'],
      dtype='object')

In [56]:
df_nn['isFraud'] = df_nn['isFraud'].astype('category')

SettingWithCopyError: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

In [24]:
df_nn['isFraud']

57195     0
312877    0
195195    1
21600     0
12253     0
         ..
585191    0
582517    0
493713    0
526363    0
500247    0
Name: isFraud, Length: 59054, dtype: category
Categories (2, int64): [0, 1]

In [9]:
cont = ['TransactionDT', 'TransactionAmt', 'C13', 'D3', 'D2', 'D1', 'D15', 
       'D5', 'D4', 'D11', 'D8', 'C9', 'C1', 'C14', 'C2', 'C5', 'C6',  'C8',
       'C12', 'C11', 'D8', 'id_02', 'C7', 'id_01', 'D9', 'id_06', 'id_05',
       'dist1', 'dist2', 'D6', 'D12']
cat =  ['card1', 'card2', 'addr1',  'P_emaildomain', 'card5', 'M4', 
        'R_emaildomain', 'card6', 'M5', 'id_31', 'id_19', 'card4', 'M6', 
       'id_17', 'ProductCD', 'id_13', 'DeviceInfo', 'card3', 'D2_na', 'M9', 'M7',
       'DeviceType', 'D3_na', 'addr2', 'M2', 'D5_na', 'id_33', 'D4_na',
        'dist1_na', 'id_37', 'id_02_na', 'id_14', 'id_18', 'D11_na', 'D15_na']

In [10]:
assert len(df_nn.columns)==(len(cat)+len(cont)+1)==(len(xs_final.columns)+1)

In [11]:
df_nn[cat].nunique()

card1            5603
card2             500
addr1             144
P_emaildomain      60
card5              78
M4                  4
R_emaildomain      60
card6               5
M5                  3
id_31             108
id_19             335
card4               5
M6                  3
id_17              54
ProductCD           5
id_13              37
DeviceInfo        767
card3              63
D2_na               2
M9                  3
M7                  3
DeviceType          3
D3_na               2
addr2              25
M2                  3
D5_na               2
id_33             113
D4_na               2
dist1_na            2
id_37               3
id_02_na            2
id_14              21
id_18              14
D11_na              2
D15_na              2
dtype: int64

There are a number of categorical variables where the number of categories has high cardinality, so deep learning might help us in this senerio.

In [12]:
# The rows with TransactionDT <= 12579058 comprise the first 80% of the data (approximately)
cut_off = math.floor((df_nn['TransactionDT'].max() - df_nn['TransactionDT'].min())*0.8)
cut_off

12579784

In [13]:
cond = df_nn['TransactionDT'] <= cut_off
train_idx = np.where( cond)[0]
valid_idx = np.where(~cond)[0]

splits = (list(train_idx),list(valid_idx))

In [14]:
type(df_nn)

pandas.core.frame.DataFrame

In [15]:
procs_nn = [Categorify, FillMissing, Normalize]
to_nn = TabularPandas(df_nn, procs_nn, cat, cont,
                     splits=splits, y_names=dep_var)
to_nn.loss_func

ValueError: Unable to coerce to Series, length must be 31: given 30

In [58]:
dls = to_nn.dataloaders(1024)

NameError: name 'to_nn' is not defined

In [23]:
dls.xs

,R_emaildomain,M4,M5,card6,id_20,ProductCD,DeviceType,addr2,id_31,card2,isFraud,D3_na,D2_na,D5_na,D1_na,D8_na,D15_na,C5,D3,C14,C13,C4,D2,C1,C8,C2,D5,C11,C10,C12,D1,D8,D15,C6,TransactionDT,TransactionAmt
57195,0,0,0,3,0,5,0,21,0,390,0,1,2,1,1,2,1,-0.134384,9.471498,-0.123058,-0.224706,-0.061955,-0.280162,-0.089028,-0.056906,-0.090942,4.260183,-0.096037,-0.057582,-0.051114,2.468850,-0.154975,0.805232,-0.109859,-1.245707,1.240415
312877,0,1,1,3,0,5,0,21,0,168,0,2,2,1,1,2,1,-0.174292,-0.233869,-0.085467,-0.150241,-0.061955,-0.280162,-0.068799,-0.056906,-0.079297,2.774099,-0.096037,-0.057582,-0.051114,-0.590402,-0.154975,1.033601,-0.097012,0.519150,-0.328971
195195,0,1,2,3,0,5,0,21,0,317,1,1,1,1,1,2,1,-0.214200,-0.404513,-0.141853,-0.180027,-0.061955,-0.634464,-0.095771,-0.056906,-0.085119,-0.391905,-0.086453,-0.057582,-0.051114,-0.259847,-0.154975,-0.483425,-0.109859,-0.405201,-0.459771
21600,0,2,2,3,0,5,0,21,0,94,0,2,2,2,1,2,1,-0.214200,-0.233869,-0.141853,-0.232152,-0.061955,-0.280162,-0.095771,-0.056906,-0.090942,-0.246527,-0.096037,-0.057582,-0.051114,-0.590402,-0.154975,-0.760731,-0.122707,-1.448797,-0.481571
12253,0,0,0,3,0,5,0,21,0,115,0,1,1,2,1,2,2,-0.094476,-0.255200,-0.123058,-0.105562,-0.061955,2.700589,-0.089028,-0.056906,-0.085119,-0.246527,-0.076868,-0.057582,-0.051114,2.546627,-0.154975,-0.499737,-0.109859,-1.510049,-0.333549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224165,55,1,0,2,143,1,1,0,74,76,0,2,2,1,1,2,1,-0.214200,-0.233869,-0.123058,-0.224706,-0.048718,-0.280162,-0.089028,-0.038321,-0.085119,3.840203,-0.086453,-0.039016,-0.030764,-0.590402,-0.154975,-0.760731,-0.109859,-0.164251,-0.464052
322952,17,3,0,2,0,1,0,0,0,390,1,2,2,1,1,2,1,-0.214200,-0.233869,-0.160649,-0.239599,-0.048718,-0.280162,-0.095771,-0.047614,-0.085119,-0.391905,-0.086453,-0.048299,-0.030764,3.188292,-0.154975,-0.760731,-0.109859,0.582717,-0.047759
449491,0,1,2,3,0,5,0,21,0,320,0,1,1,1,1,2,1,-0.174292,-0.383183,-0.141853,-0.090669,-0.061955,3.039486,-0.095771,-0.056906,-0.090942,-0.375752,-0.096037,-0.057582,-0.051114,2.831812,-0.154975,2.115638,-0.109859,1.525638,-0.328971
343893,2,3,0,3,0,1,0,17,0,445,0,2,2,2,1,2,1,-0.214200,-0.233869,-0.141853,-0.232152,-0.048718,-0.280162,-0.095771,-0.056906,-0.090942,-0.246527,-0.096037,-0.057582,-0.040939,-0.590402,-0.154975,-0.760731,-0.109859,0.707703,-0.447118


In [24]:
learn = tabular_learner(dls, layers=[500,250], metrics=[accuracy])

In [2]:
learn.validate()

NameError: name 'learn' is not defined

In [25]:
learn.loss_func

FlattenedLoss of CrossEntropyLoss()

In [1]:
#learn.lr_find()

In [ ]:
learn.fit_one_cycle(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,time


In [417]:
def r_mse(pred,y): return round(math.sqrt(((pred-y)**2).mean()), 6)
def m_rmse(y):
    preds = np.array(learn.get_preds()[0].squeeze())
    return r_mse(y, preds)

In [418]:
m_rmse(dls.valid_ds.y)

0.045772

To get the above result I treated the classification problem isFraud=1, ~isFraud=2 as a regression problem. Used MSELoss

In [421]:
preds = learn.get_preds()[0]
preds

tensor([[1.0257],
        [2.0897],
        [1.0185],
        ...,
        [1.0248],
        [1.0557],
        [0.9901]])

In [426]:
learn.get_preds()[0]>1.1

tensor([[False],
        [ True],
        [False],
        ...,
        [False],
        [False],
        [False]])

In [427]:
learn.get_preds()[1]>1.1

tensor([[False],
        [ True],
        [False],
        ...,
        [False],
        [False],
        [False]])

In [428]:
roc_auc_score(learn.get_preds()[1]>1.1, learn.get_preds()[0]>1.1)

0.9871521380781537

In [429]:
dls.xs.columns


Index(['R_emaildomain', 'M4', 'M5', 'card6', 'id_20', 'ProductCD',
       'DeviceType', 'addr2', 'id_31', 'card2', 'isFraud', 'D3_na', 'D2_na',
       'D5_na', 'D1_na', 'D8_na', 'D15_na', 'C5', 'D3', 'C14', 'C13', 'C4',
       'D2', 'C1', 'C8', 'C2', 'D5', 'C11', 'C10', 'C12', 'D1', 'D8', 'D15',
       'C6', 'TransactionDT', 'TransactionAmt'],
      dtype='object')

## Full Data

In [ ]:
to = (path/'to2.pkl').load()

In [ ]:
df_nn = to[list(xs_final.columns) + [dep_var]]
assert len(xs_final.columns)==(len(df_nn.columns)-1)
df_nn.head()

In [ ]:
df_nn.columns